<a href="https://colab.research.google.com/github/remre/StriveSchool-ai/blob/main/w6_deepl/sol/d6/TabularDataLoaderPytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook created for load data compatible with Neural Network class by dataloader

In [86]:

#first starts with importing libraries
import torch # IN somehow kind of a numpy 
from torch import nn
from torch import optim

import torchvision # package consists of popular datasets, model architectures, and common image transformations for computer vision
from torch.utils.data import Dataset, DataLoader, random_split #At the heart of PyTorch data loading utility is the torch.utils.data.DataLoader class.
                                                # It represents a Python iterable over a dataset, with support for
import torch.nn.functional as F
import numpy as np 
import math
import pandas as pd


To create this class we need two key method



*  __len__: it has to return the number of rows of the input table.
*  __getitem__: it has to return a single observation, including both the independent variables and the dependent variable. 



For the case our csv file is audi.csv we will investigate it closely in below

In [66]:
class CustomDataset(Dataset):
    def __init__(self, csv_file ='audi.csv' ):
       df = pd.read_csv(csv_file)
       self.categorical = ['transmission','fuelType','model']
       self.target = ['price']

       self.car_frame = pd.get_dummies(df, prefix=self.categorical)
       self.X = self.car_frame.drop(self.target, axis=1)
       
       
       self.y = (df.price.values <= 15000).astype("int") #here we change the price value and keep it as 1 or 0
        
    def __getitem__(self, idx):
        # we want to be index like dataset[index]
        # to get the index-th batch
        if isinstance(idx, torch.Tensor):
            idx = idx.tolist()

        return [self.X.iloc[idx].values, self.y[idx]]
    def __len__(self):
        # to retrieve the total samples by doing len(dataset)
        return len(self.car_frame)

In [31]:
class CustomDatasett(Dataset):
    def __init__(self, csv_file ='audi.csv' ):
       df = pd.read_csv(csv_file)
       self.categorical = ['transmission','fuelType','model']
       self.target = 'price'

       self.car_frame = pd.get_dummies(df, prefix=self.categorical)
       self.X = self.car_frame.drop(self.target, axis=1)
       # the target or y  is price but we will change it little bit and the rest of the columns will represent features 'X'
       
       
       self.y = self.car_frame[self.target]#here we keep price as continuous value 
        
    def __getitem__(self, idx):
        # we want to be index like dataset[index]
        # to get the index-th batch
        # Convert idx from tensor to list due to pandas bug (that arises when usingp pytorch's random_slit)
        if isinstance(idx, torch.Tensor):
            idx = idx.tolist()

        return [self.X.iloc[idx].values, self.y[idx]]
    def __len__(self):
        # to retrieve the total samples by doing len(dataset)
        return len(self.car_frame)

In [67]:
dataset = CustomDataset()
dataloader = DataLoader(dataset=dataset, batch_size=4, shuffle=True)
##DataLoader(dataset, batch_size=1, shuffle=False, sampler=None,
           #batch_sampler=None, num_workers=0, collate_fn=None,
           #pin_memory=False, drop_last=False, timeout=0,
          # worker_init_fn=None, *, prefetch_factor=2,
          # persistent_workers=False)

In [81]:
dataiter = iter(dataloader)
data = dataiter.next()
data


[tensor([[2.0170e+03, 4.9221e+04, 1.4500e+02, 5.5400e+01, 1.4000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          1.0000e+00],
         [2.0190e+03, 1.0000e+03, 1.4500e+02, 3.8200e+01, 2.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00,
          0.0000e+00],
         [

In [95]:

df = pd.read_csv('audi.csv')

In [96]:
df

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,A1,2017,12500,Manual,15735,Petrol,150,55.4,1.4
1,A6,2016,16500,Automatic,36203,Diesel,20,64.2,2.0
2,A1,2016,11000,Manual,29946,Petrol,30,55.4,1.4
3,A4,2017,16800,Automatic,25952,Diesel,145,67.3,2.0
4,A3,2019,17300,Manual,1998,Petrol,145,49.6,1.0
...,...,...,...,...,...,...,...,...,...
10663,A3,2020,16999,Manual,4018,Petrol,145,49.6,1.0
10664,A3,2020,16999,Manual,1978,Petrol,150,49.6,1.0
10665,A3,2020,17199,Manual,609,Petrol,150,49.6,1.0
10666,Q3,2017,19499,Automatic,8646,Petrol,150,47.9,1.4


In [87]:
# Split into training and test
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
trainset, testset = random_split(dataset, [train_size, test_size])

# Dataloaders
trainloader = DataLoader(trainset, batch_size=200, shuffle=True)
testloader = DataLoader(testset, batch_size=200, shuffle=False)


As 

In [73]:
class NeuralNetworkk(nn.Module):
    def __init__(self, input_dim, num_hidden1, num_hidden2):
        super().__init__() 
        #self.conv1 = nn.Conv2d(input_dim,num_hidden1,kernel_size = 2)
        self.fc1 = nn.Linear(input_dim, num_hidden1)
        self.drop = nn.Dropout(p = 0.25)
        self.fc2 = nn.Linear(num_hidden1, num_hidden2)
        self.fc3 = nn.Linear(num_hidden2, 2)
        
    def forward(self,x):
      #for layer 1 
      #x = x.flatten()
      #x = self.conv1(x)
      x = self.fc1(x)
      x = F.relu(x)
      x = self.fc2(x)
      x = F.relu(x)
      x = self.drop(x)
      x = self.fc3(x)
      #output = F.softmax(x, dim=1)
      return x
model = NeuralNetworkk(37, 100,640)

In [74]:
def get_num_correct(preds,labels):
  return preds.argmax(dim=1).eq(labels).sum().item()

In [75]:
torch.set_grad_enabled(True)
torch.set_printoptions(linewidth=120)
optimizer = optim.Adam(model.parameters(),lr = 0.01)

In [94]:


for epoch in range(10):
  total_loss = 0 
  total_correct = 0 
  for batch in dataloader:
    images, labels = batch 
    images  = images.type(torch.float32)


    preds = model(images)
    
    loss = F.cross_entropy(preds,labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    total_loss += loss.item()
    total_correct += get_num_correct(preds,labels)

print('epoch: ',epoch, 'total_correct: ', total_correct/len(dataloader), 'loss: ' , total_loss)

epoch:  9 total_correct:  2.9996250468691414 loss:  1317.1692179241759


### Exercise

Create a dataset with three classes of images (choose the classes and download your own images. You don't need to train, so around 10 images per class will be enough).

Then visualize the images with the help of the `imshow` helper function provided below:

In [ ]:
train = pd.DataFrame({"path": ["/content/drive/MyDrive/dataAI/pytorchdataloadjpg/0805personali01.jpg", "/content/drive/MyDrive/dataAI/pytorchdataloadjpg/10comm-decarlo.jpg", "/content/drive/MyDrive/dataAI/pytorchdataloadjpg/1198_0_861.jpg"], "label": [0, 1, 2] })

In [ ]:
train

,path,label
0,/content/drive/MyDrive/dataAI/pytorchdataloadj...,0
1,/content/drive/MyDrive/dataAI/pytorchdataloadj...,1
2,/content/drive/MyDrive/dataAI/pytorchdataloadj...,2


In [ ]:
train.to_csv("/content/drive/MyDrive/dataAI/train.csv")

In [ ]:
from PIL import Image
class CustomDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        super().__init__()
        df = pd.read_csv(csv_file)
        self.paths = df.path.values
        self.labels = df.label.values
    def __getitem__(self, index):
        # we want to be index like dataset[index]
        # to get the index-th batch
        img = Image.open(self.paths[index]).convert("RGB")
        if self.transform is not None:
            img = self.transform(img)
        return img, self.labels[index]
    
    def __len__(self):
        # to retrieve the total samples by doing len(dataset)
        return len(self.paths)

In [ ]:
def imshow(image, ax=None, title=None, normalize=False):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    image = image.numpy().transpose((1, 2, 0))

    if normalize:
        mean = np.array([0.485, 0.456, 0.406])
        std = np.array([0.229, 0.224, 0.225])
        image = std * image + mean
        image = np.clip(image, 0, 1)

    ax.imshow(image)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.tick_params(axis='both', length=0)
    ax.set_xticklabels('')
    ax.set_yticklabels('')

    return ax


# Run this to test your data loaders
images, labels = next(iter(trainloader))
imshow(images[0], normalize=False)